# **Basic Workflow creation**

In [ ]:
from llama_index.core.workflow import StartEvent,StopEvent,Workflow,step

class MyWorkflow(Workflow):
    @step 
    async def my_step(self, ev:StartEvent) ->StopEvent:
        return StopEvent(result="Hello world!")
    
w = MyWorkflow(timeout=10)
result = await w.run()
result


'Hello world!'

# **Connecting multiple steps**

In [3]:
from llama_index.core.workflow import Event

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev:StartEvent) ->ProcessingEvent:
        return ProcessingEvent(intermediate_result="Step 1 completed")
    
    @step
    async def step_two(self, ev: ProcessingEvent) -> StopEvent:
        return StopEvent(result=f"Step 2 result: {ev.intermediate_result}")
    
w = MultiStepWorkflow(timeout=10)

result = await w.run()

result

'Step 2 result: Step 1 completed'

# **loops and branches**

In [14]:
import random 
class ProcessingEvent(Event):
    intermediate_result: str


class LoopEvent(Event):
    loop_output: str


class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> ProcessingEvent | LoopEvent:
        if random.randint(0, 1) == 0:
            print("Bad thing happened")
            return LoopEvent(loop_output="Back to step one.")
        else:
            print("Good thing happened")
            return ProcessingEvent(intermediate_result="First step complete.")

    @step
    async def step_two(self, ev: ProcessingEvent ) -> StopEvent:
        # Use the intermediate result
        final_result = f"Finished processing: {ev.intermediate_result}"
        return StopEvent(result=final_result)


w = MultiStepWorkflow(verbose=False)
result = await w.run()
result


Bad thing happened
Good thing happened


'Finished processing: First step complete.'

# **Drawing workflows**

In [15]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(w)

<class 'NoneType'>
<class '__main__.ProcessingEvent'>
<class '__main__.LoopEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
workflow_all_flows.html


# **State management**

In [16]:
from llama_index.core.workflow import Event,Context
from llama_index.core.agent.workflow import ReActAgent

class ProcessingEvent(Event):
    intermediate_result: str

class MultiStepWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent, ctx: Context) -> ProcessingEvent:
        await ctx.set("Query","What is captial of France?")
        return ProcessingEvent(intermediate_result="Step 1 completed")
    
    @step
    async def step_two(self, ev: ProcessingEvent, ctx: Context) -> StopEvent:
        query = await ctx.get("Query")
        print(f"Query:{query}")
        final_result = f"Finished processing:{ev.intermediate_result}"
        return StopEvent(result=final_result)
w = MultiStepWorkflow(timeout=10)

result = await w.run()

result

Query:What is captial of France?


'Finished processing:Step 1 completed'

# **Multi Agent**

In [18]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow, ReActAgent

# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct")

# we can pass functions directly without FunctionTool -- the fn/docstring are parsed for the name/description
multiply_agent = ReActAgent(
    name="multiply_agent",
    description="Is able to multiply two integers",
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    tools=[multiply], 
    llm=llm,
)

addition_agent = ReActAgent(
    name="add_agent",
    description="Is able to add two integers",
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools=[add], 
    llm=llm,
)

# Create the workflow
workflow = AgentWorkflow(
    agents=[multiply_agent, addition_agent],
    root_agent="multiply_agent"
)

# Run the system
response = await workflow.run(user_msg="Can you add 5 and 3?")

In [19]:
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='5 + 3 = 8')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'add_agent', 'reason': 'The user wants to add two numbers and the add_agent is better suited for this task.'}, tool_id='de40d2f9-abe8-488c-a5b9-36b09e58ef92', tool_output=ToolOutput(content='Agent add_agent is now handling the request due to the following reason: The user wants to add two numbers and the add_agent is better suited for this task..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'to_agent': 'add_agent', 'reason': 'The user wants to add two numbers and the add_agent is better suited for this task.'}}, raw_output='Agent add_agent is now handling the request due to the following reason: The user wants to add two numbers and the add_agent is better suited for this task..\nPlease continue with the cu

# **Multi agent with context**

In [21]:
from llama_index.core.workflow import Context

async def add(ctx: Context, a: int, b: int) ->int:
    """Add two numbers."""
    curr_state = await ctx.get("state")
    curr_state["num_fn_calls"] +=1
    await ctx.set("state",curr_state)
    return a + b

async def multiply(ctx: Context, a: int, b: int) -> int:
    """Multiply two numbers."""
    curr_state = await ctx.get("state")
    curr_state["num_fn_calls"] +=1
    await ctx.set("state", curr_state)
    return a * b
multiply_agent = ReActAgent(
    name="multiply_agent",
    description="Is able to multiply two integers",
    system_prompt="A helpful assistant that can use a tool to multiply numbers.",
    tools=[multiply], 
    llm=llm,
)

addition_agent = ReActAgent(
    name="add_agent",
    description="Is able to add two integers",
    system_prompt="A helpful assistant that can use a tool to add numbers.",
    tools=[add], 
    llm=llm,
)

workflow = AgentWorkflow(
    agents=[multiply_agent,addition_agent],
    root_agent="multiply_agent",
    initial_state={"num_fn_calls":0},
    state_prompt="Current state: {state}. User message :{msg}"
)

ctx = Context(workflow)

response = await workflow.run(user_msg="Can you add 5 and 3?",ctx=ctx)

state = await ctx.get("state")
state

{'num_fn_calls': 1}

In [ ]:
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='The sum of 5 and 3 is 8.')]), tool_calls=[ToolCallResult(tool_name='handoff', tool_kwargs={'to_agent': 'add_agent', 'reason': 'The user wants to add two numbers, and the add_agent is better suited for this task.'}, tool_id='23c44b98-8f9e-4d4d-9e82-9b426a47fd7b', tool_output=ToolOutput(content='Agent add_agent is now handling the request due to the following reason: The user wants to add two numbers, and the add_agent is better suited for this task..\nPlease continue with the current request.', tool_name='handoff', raw_input={'args': (), 'kwargs': {'to_agent': 'add_agent', 'reason': 'The user wants to add two numbers, and the add_agent is better suited for this task.'}}, raw_output='Agent add_agent is now handling the request due to the following reason: The user wants to add two numbers, and the add_agent is better suited for this task..\nPlease c